<h1 style=" text-align:center; color:Blue; font-size:40px;"> <u><b>Cat Breeds prediction with InceptionV3</b></u> </h1>

<p style="text-align:center; ">
<img src="https://images.theconversation.com/files/168121/original/file-20170505-1693-ymh4bc.jpg" style='width: 800px;'>
</p>



## 1. Reading Modules


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from keras.optimizers import Adam, SGD, RMSprop
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
from scipy import ndimage, misc
import skimage
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Sequential
from keras.layers.pooling import GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Dropout



In [ ]:
filelist  = []

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        filelist.append (os.path.join(dirname, filename))
        


In [ ]:
len(filelist)

In [ ]:
filelist

### Taking only those 10 labels here.

In [ ]:
labels_needed = ['Bombay', 'Calico', 'Burmese', 'Himalayan',
'Munchkin', 'Ragdoll', 'Siberian', 'British Shorthair', 'Russian Blue', 
                 'Dilute Calico']

In [ ]:
Filepaths   = []
labels = []

for image_file in filelist:
    label = image_file.split(os.path.sep)[-2]
    if label in labels_needed:

        Filepaths.append(image_file)
        labels.append(label)
    


In [ ]:
set(labels)

In [ ]:
len(Filepaths), len(labels)

### Creating a dataframe with file paths and the labels for them

In [ ]:
df = pd.DataFrame( list( zip (Filepaths, labels) ), columns = ['Filepath', 'Labels'] )
df

In [ ]:
from sklearn.utils import shuffle
df = (df.sample(frac = 1).reset_index()).drop(columns = 'index')
df


### Displaying first 12 pictures

In [ ]:
f,a = plt.subplots(nrows=4, ncols=3,figsize=(13, 7),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(a.flat):
    ax.imshow(plt.imread(df.Filepath[i]))
    ax.set_title(df.Labels[i])
    
plt.tight_layout()
plt.show()

In [ ]:
ax=pd.value_counts(df['Labels'],ascending=True).plot(kind='barh',
                                                       fontsize="40",
                                                       title="Distribution Of classes",
                                                       figsize=(15,8))
ax.set(xlabel="Images per class", ylabel="Classes")
ax.xaxis.label.set_size(40)
ax.yaxis.label.set_size(40)
ax.title.set_size(60)
plt.show()

### Checking for class imbalance

In [ ]:
df.Labels.value_counts()

* Class imbalance present.

### Splitting the data And Creating data generator

In [ ]:
train_ratio = .75
validation_ratio = 0.10
test_ratio = 0.25

train, test = train_test_split(df, test_size = test_ratio )
val, test = train_test_split(test, test_size=test_ratio/(test_ratio + validation_ratio))

In [ ]:
img_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=30, 
                                   width_shift_range=0.2,
                                   height_shift_range=0.2, 
                                   horizontal_flip = 'true')
   

x_train =  img_datagen.flow_from_dataframe(dataframe = train,  x_col='Filepath', y_col='Labels',  target_size=(299, 299), shuffle=False, batch_size=10, seed=10)
x_val = img_datagen.flow_from_dataframe(dataframe = val,  x_col='Filepath', y_col='Labels',  target_size=(299, 299), shuffle=False, batch_size=10, seed=10)
x_test = img_datagen.flow_from_dataframe(dataframe = test,  x_col='Filepath', y_col='Labels',  target_size=(299, 299), shuffle=False, batch_size=10, seed=10)

In [ ]:
x_train

### Modelling

In [ ]:

i_model = InceptionV3(weights= 'imagenet', include_top=False, input_shape=(299, 299, 3))

In [ ]:
for layer in i_model.layers:
    layer.trainable = False
    
i_model.summary() 

In [ ]:
model = Sequential()
model.add(i_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Dense(10, activation = 'softmax'))
model.summary()

In [ ]:
model.compile(optimizer = SGD(),
             loss="categorical_crossentropy",
             metrics=["accuracy"])

In [ ]:
history = model.fit(x_train, validation_data = x_val,steps_per_epoch = 175,validation_steps = 44,
                epochs = 15, verbose = 2)

### Predicting on test data

In [ ]:
predictions = model.predict(x_test)
predictions = np.argmax(predictions, axis=1)
predictions

In [ ]:
labels = x_train.class_indices
labels

In [ ]:
test["Labels"].replace({"Bombay": 0,'British Shorthair': 1,
 'Burmese': 2,
 'Calico': 3,
 'Dilute Calico': 4,
 'Himalayan': 5,
 'Munchkin': 6,
 'Ragdoll': 7,
 'Russian Blue': 8,
 'Siberian': 9}, inplace = True)


# Evaluating the test data

## Test Accuracy

In [ ]:
test_accuracy = model.evaluate(x_test)[1] * 100
print('Test accuracy is : ',test_accuracy, '%' )

## Confusion Matrix

In [ ]:
confusion_matrix(test.Labels , predictions)


## F1 Score

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
print('F1 score is',f1_score(test.Labels, predictions, average = 'weighted') *100, "%")


## ROC - AUC Score

In [ ]:
predicted_probab =model.predict_proba(x_test)
predicted_probab

In [ ]:
print("ROC- AUC score is", roc_auc_score( test.Labels, predicted_probab, multi_class='ovr')*100)

# Conclusion

The model is able to classify the images with a roughly 65% accuracy, however, ROC- AUC score goes upto 90.5 %.


## Please do share you feedback and upvote if you likemy work! :)